# EfficientNet-B4 Processed Merged Gamma with extra cross entropy Weight to handle imbalance


Here we run experiment with ODIR-2019 Gamma enhanced Processed images and additional training images from Ocular Diseases DataSet
except for Normal which we leave the ODIR-2019 original images as it is

In [1]:
data_dir = './ODIR-2019/YOLO/processed_512g_merged'  # Your dataset path
model_name = 'efficientnet-b4'
saved_model_name = 'efficientnet-b4-odir-2019-pmgw.pth'
wandb_project = "odir-2019"
wandb_run_name = 'efficientnet-b4_pmgw'

The following codes are generic or the same across experiments

In [2]:
%%capture
!pip install efficientnet-pytorch wandb torch torchvision scikit-learn seaborn tqdm


In [3]:
import sys
sys.path.append('.')

from efficient_net import EfficientNetTrainer, ODIRFolderDataset

In [4]:
from util import get_train_device

device_for_training = get_train_device()

Using NVIDIA GPU (CUDA)


In [5]:
# demonstrate weighted loss result
train=ODIRFolderDataset(
            root_dir=data_dir,
            split='train',
            image_size=512
        )
train.get_class_weights()

Found 9478 images in train split across 8 classes

Class distribution in train split:
  normal: 2252 samples (23.8%)
  diabetes: 2790 samples (29.4%)
  glaucoma: 1574 samples (16.6%)
  cataract: 724 samples (7.6%)
  ageing: 724 samples (7.6%)
  hypertension: 394 samples (4.2%)
  myopia: 475 samples (5.0%)
  other: 545 samples (5.8%)


[0.5260879218472468,
 0.4246415770609319,
 0.752700127064803,
 1.6363950276243093,
 1.6363950276243093,
 3.0069796954314723,
 2.4942105263157894,
 2.1738532110091744]

In [6]:
import wandb
config = {
    'data_dir': data_dir,
    'model_name': model_name,
    'num_classes': 8,
    'image_size': 512,
    'batch_size': 16,  # Smaller for Jupyter
    'epochs': 30,
    'lr': 0.001,
    'weight_decay': 1e-4,
    'optimizer': 'adamw',
    'scheduler': 'cosine',
    'fine_tune': True,
    'unfreeze_blocks': 3,
    'label_smoothing': 0.1,
    'num_workers': 2,  # Lower for Jupyter
    'use_amp': True,
    'seed': 42,
    'use_weight': True  # use reverse balanced weight for our cross entropy loss
}
wandb.init(project=wandb_project, name=wandb_run_name, config=config)
trainer = EfficientNetTrainer(config)
trainer.train()

wandb: Currently logged in as: raymond-samalo (samalo) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Using device: cuda
Loading efficientnet-b4 with 8 classes...
Loaded pretrained weights for efficientnet-b4
Loading datasets from ./ODIR-2019/YOLO/processed_512g_merged...
Found 9478 images in train split across 8 classes

Class distribution in train split:
  normal: 2252 samples (23.8%)
  diabetes: 2790 samples (29.4%)
  glaucoma: 1574 samples (16.6%)
  cataract: 724 samples (7.6%)
  ageing: 724 samples (7.6%)
  hypertension: 394 samples (4.2%)
  myopia: 475 samples (5.0%)
  other: 545 samples (5.8%)
Found 627 images in val split across 8 classes

Class distribution in val split:
  normal: 282 samples (45.0%)
  diabetes: 160 samples (25.5%)
  glaucoma: 28 samples (4.5%)
  cataract: 26 samples (4.1%)
  ageing: 27 samples (4.3%)
  hypertension: 13 samples (2.1%)
  myopia: 23 samples (3.7%)
  other: 68 samples (10.8%)
Found 627 images in test split across 8 classes

Class distribution in test split:
  normal: 282 samples (45.0%)
  diabetes: 160 samples (25.5%)
  glaucoma: 28 samples (4.5%

Validation: 100%|██████████| 40/40 [00:29<00:00,  1.36it/s, Loss=1.9777, Acc=48.96%]



📊 Epoch 1 Summary:
Train - Loss: 1.6729, Acc: 48.51%, F1: 0.5090
Val   - Loss: 1.9777, Acc: 48.96%, F1: 0.4505
Learning Rate: 9.97e-05
  Time - Epoch: 301.5s, Inference: 16.3ms
Model saved to /home/ray/Projects/ITI123-GenerativeAIDeepLearningProject/wandb/run-20260224_103922-bfku3z2y/files/best_model.pth
✅ New best model saved with F1: 0.4505

Epoch 2/30


Validation: 100%|██████████| 40/40 [00:29<00:00,  1.35it/s, Loss=1.9196, Acc=50.08%]


...

socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.


Error in callback <bound method _WandbInit._post_run_cell_hook of <wandb.sdk.wandb_init._WandbInit object at 0x799a94523ce0>> (for post_run_cell):


socket.send() raised exception.


BrokenPipeError: [Errno 32] Broken pipe

socket.send() raised exception.


In [ ]:
# cell 6: Test
test_loss, test_acc, test_f1 = trainer.test()
print(f"Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}, Test F1: {test_f1:.4f}")

socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.


Error in callback <bound method _WandbInit._pre_run_cell_hook of <wandb.sdk.wandb_init._WandbInit object at 0x799a94523ce0>> (for pre_run_cell):


socket.send() raised exception.
socket.send() raised exception.


BrokenPipeError: [Errno 32] Broken pipe

socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.


...

Error in callback <bound method _WandbInit._post_run_cell_hook of <wandb.sdk.wandb_init._WandbInit object at 0x799a94523ce0>> (for post_run_cell):


socket.send() raised exception.


BrokenPipeError: [Errno 32] Broken pipe

socket.send() raised exception.
socket.send() raised exception.


: 

In [ ]:
import os
final_model_path = os.path.join('ODIR-2019/results/', saved_model_name)
trainer.save_model(final_model_path)

In [ ]:
wandb.finish()